In [1]:
import argparse
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
import sys
sys.path.append(('../'))
sys.path.append(('../../'))
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import json
from typing import Dict
import torch
from torch.nn import CrossEntropyLoss
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from task_vector import TaskVector

from watermarks.kgw.watermark_processor import WatermarkDetector
from watermarks.aar.aar_watermark import AarWatermarkDetector
from watermarks.watermark_types import WatermarkType
import logging


/remote-home/miintern1/anaconda3/envs/watermark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
watermark_residuals = torch.load("/remote-home/miintern1/watermark-learnability/data/c4/watermark_residuals.pt", map_location='cpu')
vanilla_residuals = torch.load("/remote-home/miintern1/watermark-learnability/data/c4/vanilla_residuals.pt", map_location='cpu')

/remote-home/miintern1/anaconda3/envs/watermark/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

### Load in Dataset

In [2]:
max_length = 250
min_length = 250
num_samples = 512
batch_size = 16
device = 'cpu'
dataset = load_dataset("allenai/c4", "realnewslike", split="validation", streaming=False)

In [4]:
dataset

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 13863
})

In [6]:
def filter_length(example):
        return len(tokenizer(example['text'], truncation=True, max_length=max_length)["input_ids"]) >= min_length

def encode(examples):
    trunc_tokens = tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)
    # Examples are truncated to max_length, which comprises the possible generation prompt and the text to be generated
    examples["text"] = tokenizer.batch_decode(trunc_tokens["input_ids"], skip_special_tokens=True)
    prompt = tokenizer(
        examples["text"], truncation=True, padding=True, max_length=50, return_tensors="pt",
    ).to(device)
    examples["prompt_text"] = tokenizer.batch_decode(prompt["input_ids"], skip_special_tokens=True)
    examples["input_ids"] = prompt["input_ids"]
    examples["attention_mask"] = prompt["attention_mask"]
    examples["text_completion"] = tokenizer.batch_decode(
        trunc_tokens["input_ids"][:, 50:], skip_special_tokens=True
    )
    return examples

dataset = dataset.filter(filter_length)
# Set how many samples will be skipped
dataset = dataset.map(encode, batched=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size)

Map: 100%|██████████| 10498/10498 [00:06<00:00, 1686.28 examples/s]


In [8]:
prompts = []
human_text = []
prompt_text = []
full_human_text = []
for batch in dataloader:
    if len(human_text) >= num_samples:
        break
    if (type(batch["input_ids"]) == list):
        batch["input_ids"] = torch.stack(batch["input_ids"], dim=1).to(device)
    if (type(batch["attention_mask"]) == list):
        batch["attention_mask"] = torch.stack(batch["attention_mask"], dim=1).to(device)
    prompts.append(batch)
    human_text.extend(batch["text_completion"])
    prompt_text.extend(batch["prompt_text"])
    full_human_text.extend(batch["text"])
human_text = human_text[:num_samples]
prompt_text = prompt_text[:num_samples]
full_human_text = full_human_text[:num_samples]
raw_input = {
    "prompts": prompts,
    "human_text": human_text,
    "prompt_text": prompt_text,
    "full_human_text": full_human_text
}

In [9]:
block_size = 512
batch_size = 8
# Initialize lists to hold the tokenized and grouped data
all_input_ids = []
all_attention_masks = []
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.pad_token = tokenizer.eos_token
# Tokenize and concatenate texts
for example in tqdm(dataset):
    text = example["text"]
    # Tokenize the text
    
    tokenized_text = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=block_size)
    
    # Add the tokenized text to the lists
    all_input_ids.append(tokenized_text['input_ids'].squeeze().tolist())
    all_attention_masks.append(tokenized_text['attention_mask'].squeeze().tolist())
print(f"{len(all_input_ids)}=")

# Flatten the lists
from itertools import chain
all_input_ids = list(chain(*all_input_ids))
all_attention_masks = list(chain(*all_attention_masks))

# Ensure the total length is a multiple of block_size
total_length = (len(all_input_ids) // block_size) * block_size
all_input_ids = all_input_ids[:total_length]
all_attention_masks = all_attention_masks[:total_length]

# Split the tokenized texts into chunks of block_size
grouped_input_ids = [all_input_ids[i:i + block_size] for i in range(0, total_length, block_size)]
grouped_attention_masks = [all_attention_masks[i:i + block_size] for i in range(0, total_length, block_size)]

# Create batches of size batch_size
batched_input_ids = [grouped_input_ids[i:i + batch_size] for i in range(0, len(grouped_input_ids), batch_size)]
batched_attention_masks = [grouped_attention_masks[i:i + batch_size] for i in range(0, len(grouped_attention_masks), batch_size)]


batched_input_ids = [torch.tensor(batch) for batch in batched_input_ids]
batched_attention_masks = [torch.tensor(batch) for batch in batched_attention_masks]

# Example: Inspect the first batch
print("First batch input IDs:", batched_input_ids[0].shape)
print("First batch attention masks:", batched_attention_masks[0].shape)


100%|██████████| 10498/10498 [00:10<00:00, 981.26it/s] 


10498=


In [55]:
from itertools import chain
batch_size=16
block_size=512
def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
        # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

In [67]:
len(prompts)

32

### Use OpenWebText Dataset

In [10]:
raw_datasets = load_dataset(
            'Skylion007/openwebtext',
        )

### Initialize Models

In [11]:
hf_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",device_map = 'cpu')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [01:26<00:00, 43.01s/it]


In [12]:
watermark_config = {
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta2":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta2":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k2-gamma0.25-delta2":{"type": "kgw", "k": 2, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_2", "kgw_device": "cpu"},
}
watermark_name, watermark_config = watermark_config.popitem()
watermarked_model = AutoModelForCausalLM.from_pretrained(watermark_name)   

Loading checkpoint shards: 100%|██████████| 3/3 [02:56<00:00, 58.91s/it]


In [31]:
def hook_fn(module, input, output):
    print(f"Hook called for {module}")
    return output


# Register the hook to the desired layer. For example, to capture the output of the first transformer block:
layer_outputs = {}

def get_layer_output_hook(layer_name):
    def hook_fn(module, input, output):
        layer_outputs[layer_name] = output
    return hook_fn


layer_name = "model.layers.31"  # This might need to be adjusted based on your model architecture
layer = dict([*hf_model.named_modules()])[layer_name]
layer.register_forward_hook(get_layer_output_hook(layer_name))

# for i in range(0,1):  # Change the range to match the number of layers you want to capture
#     layer_name = f"model.layers.{i}"  # Adjust the layer name/path as needed
#     layer = dict([*hf_model.named_modules()])[layer_name]
#     print(layer)
#     layer.register_forward_hook(get_layer_output_hook(layer_name))


import torch

# Example input
input_text = "This is a test."
inputs = tokenizer(input_text, return_tensors="pt")

# Move inputs to the device if necessary
inputs = {k: v.to(hf_model.device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = hf_model(**inputs)

# Retrieve the intermediate embeddings
intermediate_embeddings = layer_outputs[layer_name]
print(intermediate_embeddings[0].shape)
layer_outputs.keys()

torch.Size([1, 6, 4096])


dict_keys(['model.layers.0', 'model.layers.1', 'model.layers.2', 'model.layers.3', 'model.layers.4', 'model.layers.5', 'model.layers.6', 'model.layers.7', 'model.layers.8', 'model.layers.9', 'model.layers.10', 'model.layers.11', 'model.layers.12', 'model.layers.13', 'model.layers.14', 'model.layers.15', 'model.layers.16', 'model.layers.17', 'model.layers.18', 'model.layers.19', 'model.layers.20', 'model.layers.21', 'model.layers.22', 'model.layers.23', 'model.layers.24', 'model.layers.25', 'model.layers.26', 'model.layers.27', 'model.layers.28', 'model.layers.29', 'model.layers.30', 'model.layers.31'])

### Genrate watermarked logit and original logit

In [17]:
batch = prompts[0]
print(batch.keys())
DO_SAMPLE = True
temperature=1.0
top_p=0.9
top_k=0
print(batch['input_ids'].shape)

dict_keys(['text', 'timestamp', 'url', 'prompt_text', 'input_ids', 'attention_mask', 'text_completion'])
torch.Size([16, 50])


In [14]:
vanilla_output = hf_model.generate(
                            input_ids=batch["input_ids"],
                            attention_mask=batch["attention_mask"],
                            do_sample=DO_SAMPLE,
                            min_new_tokens=200,
                            max_new_tokens=200,
                            temperature=temperature,
                            top_p=top_p,
                            top_k=top_k,
                            pad_token_id=tokenizer.eos_token_id,
                        )



In [15]:
vanilla_output_logit  = hf_model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

NameError: name 'batch' is not defined

In [46]:
# Convert logits to token IDs by taking the argmax
from pprint import pprint
predicted_token_ids = torch.argmax(vanilla_output_logit['logits'], dim=-1)
print(predicted_token_ids.shape)
# Decode token IDs to text
predicted_text = tokenizer.decode(predicted_token_ids[1][-1], skip_special_tokens=True)
pprint(predicted_text)

torch.Size([16, 50])
're'


In [16]:
for input_ids, attention_mask in zip(batched_input_ids, batched_attention_masks):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    with torch.no_grad():
        vanilla_output_logit = hf_model(input_ids=input_ids, attention_mask=attention_mask).logits
    break
    

RuntimeError: expand(torch.FloatTensor{[8, 8, 1, 512, 512]}, size=[8, 1, 512, 512]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)

In [ ]:
print(vanilla_output_logit.shape)

torch.Size([16, 512, 32000])


In [1]:
tokenizer.decode(batch['input_ids'][1], skip_special_tokens=True)

NameError: name 'tokenizer' is not defined

In [44]:
batch['text'][1]

'Belying expectations, Prasar Bharti has earned only Rs 58.19 crore (Rs 581.9 million) as revenue during the Commonwealth Games last month.\nThe gross revenue earned by PB on account of telecasting/broadcasting of advertisements on Doordarshan channel and All India Radio during coverage of the Commonwealth Games is Rs 58.17 crore, Minister of State for Information and Broadcasting S Jagathrakshakan informed the Lok Sabha on Tuesday.\nWhile AIR earned Rs 2.18 crore (Rs 21.8 million), Doordarshan garnered Rs 55.99 crore (Rs 559.9 million) as revenue, he said. Prasar Bharati had earlier said it knew in advance that the recently concluded Commonwealth Games, for which Doordarshan was the official broadcaster, would not bring in huge advertising revenues.\nas a result of luke warm response from advertisers.\nNotably, DD'

In [37]:
predicted_token_ids[0].shape
batch['input_ids'][0].shape

torch.Size([50])

In [ ]:
import torch
final_result_folder = "/remote-home/miintern1/watermark-learnability/data/c4"
watermarked_logits = torch.load(final_result_folder + "watermark_logits.pt")
vanilla_logits = torch.load(final_result_folder + "vanilla_logits.pt")

In [32]:
import json
with open("/remote-home/miintern1/watermark-learnability/data/c4/watermark_vector_evaluation.json", 'r') as f:
    watermark_vector_evaluation = json.load(f)

In [75]:
coefficient_list =concatenated_array = np.concatenate((np.arange(0.0, 1.0, 0.1) , np.arange(1.0, 4.0, 0.3)))
print(coefficient_list)
print(watermark_vector_evaluation['cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta2']['0.4'].keys())

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.3 1.6 1.9 2.2 2.5 2.8 3.1
 3.4 3.7]
dict_keys(['watermarked_output', 'full_watermarked_output', 'vanilla_output', 'vanilla_scores', 'watermarked_scores', 'median_seq_rep_3', 'mean_seq_rep_3', 'list_seq_rep_3', 'total_rep_3', 'mean_perplexity', 'median_perplexity', 'perplexities'])


In [73]:
from pprint import pprint
coefficient = '1.6'
model_name = 'cygu/llama-2-7b-logit-watermark-distill-kgw-k2-gamma0.25-delta2'
print(f"Watermark Score = {watermark_vector_evaluation[model_name][coefficient]['watermarked_scores']}")
print(f"Vanilla Score = {watermark_vector_evaluation[model_name][coefficient]['vanilla_scores']}")
print(f"Perplexity = {watermark_vector_evaluation[model_name][coefficient]['mean_perplexity']}")
pprint(watermark_vector_evaluation[model_name][coefficient]['full_watermarked_output'][2])

Watermark Score = [0.016355227883506743, 1.218366051701328e-08, 0.026491784932231744, 0.17282432636257503, 0.012308539746637977, 1.3979689297990424e-06, 0.0007538752495526407, 0.0020763850313310736, 0.4296066659083557, 0.17282432636257503, 7.772294375042377e-05, 0.0033414074125787197, 0.4945272214607492, 0.0020763850313310736, 0.00014143029167768795, 0.16221293030657816, 3.1168611747939005e-07, 0.03768993880655806, 7.772294375042377e-05, 0.00526769149311491, 1.4280990971319214e-07, 1.4280990971319214e-07, 0.00813539260549211, 0.0033414074125787197, 0.00813539260549211, 2.8726776896510243e-06, 0.012308539746637977, 7.772294375042377e-05, 0.00526769149311491, 0.0014671279285978405, 0.0033414074125787197, 7.491718234060429e-19, 1.1417761815694096e-05, 0.0033414074125787197, 0.0006430814108669477, 0.026491784932231744, 4.004543805359876e-07, 0.00016911265119370079, 0.0020763850313310736, 0.0007538752495526407, 0.00044048696298454207, 4.185168928687397e-05, 6.413665629600846e-08, 5.15378203